# Clasificador de mensajes SPAM
## Técnica: <span style="color:blue">*Bayes Ingenuo*</span>

## Introducción

- ### Sobre la técnica  

Un clasificador Bayesiano ingenuo es un clasificador probabilístico fundamentado en el teorema de Bayes y una pequeña simplificación: asumimos que todas las variables involucradas en la clasificación son *independientes entre sí*; es decir, que la ocurrencia de un evento no afecta la probabilidad de ocurrencia de otro. De esta simplificación proviene el nombre de **Ingenuo**.

- ### Sobre el problema

Se plantea construir un clasificador de emails que nos diga con alta confianza si un email recibido es spam o ham (no spam). Para construir nuestro clasificador de spam-ham, haremos uso de un <a href="https://archive.ics.uci.edu/ml/machine-learning-databases/00228/">dataset</a> que contiene un total de 5574 mensajes (al momento de la investigación) y su respectiva etiqueta: **spam** o **ham**.  
Se puede obtener más información acerca del **dataset** <a href="https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection">aquí</a>

## Preparando el dataset

Tendremos algunas consideraciones acerca del dataset:
- Las letras serán convertidas a minúsculas.
- Todo número incluído en un mensaje será reemplazado por la palabra **number**.
- Las URLs serán reemplazadas por la palabra **httpaddr**.
- Consideraremos palabras que comúnmente se usan en los mensajes para evitar tomarlas en cuenta al momento de crear el diccionario. Estas fueron obtenidas desde este <a href="https://www.ranks.nl/stopwords">enlace</a>.
- Los símbolos como **, . ; @ # ? ! & \$ : _** serán retirados de los mensajes para poder trabajar mayoritariamente con las palabras.

> **Nota**: Si se desea usar el clasificador para un mensaje fuera de los mensajes de prueba (obtenidos también del dataset), tener en cuenta estas consideraciones.

## Bibliotecas necesarias necesarias

Usamos las siguientes:
- <a href="https://www.numpy.org/devdocs/reference/">**Numpy**</a> - manejo de arreglos y operaciones matemáticas como el logaritmo. (`pip install numpy`)
- <a href="https://pandas.pydata.org/pandas-docs/stable/">**Pandas**</a> - manejo de DataFrames para mejor control de datos. (`pip install pandas`)
- <a href="https://scikit-learn.org/stable/">**Scikt Learn**</a> -  para extracción de caraterísticas y selección de data de entrenamiento y prueba. (`pip install scikit-learn`)

## Obtención del diccionario

Para la creación del diccionario, se contará la cantidad de palabras presentes en los mensajes utilizados **para el entrenamiento**. Para ello, leeremos el dataset, realizaremos las conversiones necesarias (las mencionadas en **Preparando el dataset**) y dividiremos el dataset en 2 (3/4 para entrenamiento y 1/4 para prueba).

In [1]:
import pandas as pd
import collections
import numpy as np
from operator import itemgetter
import heapq
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

Declaramos la lista de palabras a evitar

In [2]:
STOP_WORDS = [
    "a", "about", "above", "across", "after", "afterwards", 
    "again", "all", "almost", "alone", "along", "already", "also",    
    "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "another", "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are", "as", "at", "be", "became", "because", "become","becomes", "becoming", "been", "before", "behind", "being", "beside", "besides", "between", "beyond", "both", "but", "by","can", "cannot", "cant", "could", "couldnt", "de", "describe", "do", "done", "each", "eg", "either", "else", "enough", "etc", "even", "ever", "every", "everyone", "everything", "everywhere", "except", "few", "find","for","found", "four", "from", "further", "get", "give", "go", "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", "however", "i", "ie", "if", "in", "indeed", "is", "it", "its", "itself", "keep", "least", "less", "ltd", "made", "many", "may", "me", "meanwhile", "might", "mine", "more", "moreover", "most", "mostly", "much", "must", "my", "myself", "name", "namely", "neither", "never", "nevertheless", "next","no", "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", "out", "over", "own", "part","perhaps", "please", "put", "rather", "re", "same", "see", "seem", "seemed", "seeming", "seems", "she", "should","since", "sincere","so", "some", "somehow", "someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "take","than", "that", "the", "their", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they",
    "this", "those", "though", "through", "throughout",
    "thru", "thus", "to", "together", "too", "toward", "towards",
    "under", "until", "up", "upon", "us",
    "very", "was", "we", "well", "were", "what", "whatever", "when",
    "whence", "whenever", "where", "whereafter", "whereas", "whereby",
    "wherein", "whereupon", "wherever", "whether", "which", "while", 
    "who", "whoever", "whom", "whose", "why", "will", "with",
    "within", "without", "would", "yet", "you", "your", "yours", "yourself", "yourselves"
]

Lectura del dataset. Lo almacenaremos en un objeto `DataFrame` de la biblioteca __pandas__ de __Python__.

In [3]:
df = pd.read_csv('C:/Users/Bryan/Documents/DjangoProjects/mysite/clasificadorSpam/SMSSpamCollection', 
                   sep='\\t', 
                   names=['etiqueta','mensaje'],engine='python')

*Cambiamos todas las letras a minúsculas*

In [4]:
df = df.apply(lambda x: x.astype(str).str.lower())

*Cambiamos los enlaces por la palabra __httpaddr__*

In [5]:
df['mensaje'] = df['mensaje'].str.replace('http\S+|www.\S+', 'httpaddr', case=False)

*Cambiamos los números por la palabra __number__*

In [6]:
df['mensaje'] = df['mensaje'].replace('\d+', ' number ', regex=True)

*Eliminamos los símbolos mencionados*

In [7]:
df['mensaje'] = df['mensaje'].replace('[,.;@#?!&$:_]+', '', regex=True)


Obtenemos mensajes para el entrenamiento y los que usaremos para la prueba.  
La función `train_test_split` dividirá las columnas `mensaje` y `etiqueta` de nuestro `DataFrame` en dos, obteniendo así un conjunto de entrenamiento y otro para la prueba.
> **Nota:** Trabajaremos con el conjunto de entrenamiento hasta que empecemos con la fase de prueba del sistema

In [8]:
msj_entrenar, msj_test, etiqueta_entrenar, etiqueta_test = train_test_split(df['mensaje'], df['etiqueta'], random_state=1)

Creamos un objeto `CountVectorizer` de la biblioteca __sklearn__ que nos ayudará con el conteo de palabras dentro de los mensajes.  
La documentación del objeto la tenemos en este <a href="https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html">link</a>

In [16]:
cont = CountVectorizer()

Realizamos el conteo de características (palabras por cada mensaje) y lo almacenamos en una matriz X

In [17]:
X = cont.fit_transform(msj_entrenar)

Obtenemos un `DataFrame` a partir de las variables anteriores: `cont.get_feature_names` nos dará la lista de palabras que fueron encontradas en **todos** los mensajes y `X.toarray` será la matriz de ocurrencias de las palabras en los mensajes.
> Nota: Las filas del DataFrame **no necesariamente coinciden** con los elementos de la lista de mensajes `msj_entrenar`

In [23]:
palabras = cont.get_feature_names()
matriz_frecuen=pd.DataFrame(data=X.toarray(), columns=palabras)

Realizaremos la sumatoria de las columnas del DataFrame anterior y obtenemos **EL TOTAL DE OCURRENCIAS DE CADA PALABRA**. Luego creamos un `DataFrame` para tener una mejor visualización de los datos
> **Nota:** `np.ravel(arr)` devuelve un arreglo significativo. En este caso, la función utilizada `arr.itemset` crea el arreglo de esta forma: `[[item1, item2, item3, ...]]`; con `np.ravel` obtendremos: `[item1, item2, item3, ...]` 

In [44]:
arr=np.ndarray(dtype=np.int32, shape=(1, len(matriz_frecuen.keys())))
for i, palabra in enumerate(matriz_frecuen):
    arr.itemset(i, sum(matriz_frecuen[palabra]))
arr = np.ravel(arr)

data=pd.DataFrame({'palabra' : palabras , 'ocurrencia':arr})

Retiramos las palabras que se encuentran en nuestro arreglo `STOP_WORDS` y obtenemos la lista de palabras ordenadas por ocurrencia.

In [30]:
data = data[~data['palabra'].isin(STOP_WORDS)]
data_ordenada=data.sort_values(by='ocurrencia', ascending=False)

Veamos con cuántas palabras contamos...

In [27]:
len(data)

6813

De esas palabras, decidimos usar solo 3000 para crear nuestro **diccionario**.

In [31]:
diccionario=data_ordenada.head(3000)

In [32]:
diccionario

,palabra,ocurrencia
4130,number,3470
842,call,451
6461,ur,290
3117,just,276
2213,free,223
4193,ok,204
3509,ltgt,193
3208,know,188
1409,day,188
3411,ll,183


# Inicio del entrenamiento

Crearemos un dataset con la data de entrenamiento

In [45]:
entrenamiento=pd.DataFrame({'mensaje' : msj_entrenar , 'etiqueta':etiqueta_entrenar})

In [47]:
len(entrenamiento)

4180

Obtenemos los mensajes dependiendo de su etiqueta: **spam** o **ham**.

In [49]:
spam=entrenamiento[entrenamiento['etiqueta'] =='spam']['mensaje']
ham=entrenamiento[entrenamiento['etiqueta'] =='ham']['mensaje']

Creamos un nuevo objeto `CountVectorizer`para el conteo de palabras. Lo usaremos en el conteo de los mensajes SPAM, así como los de HAM.

In [48]:
conts=CountVectorizer()

Esta es una variable auxiliar que contiene la totalidad de palabras dentro del diccionario.

In [50]:
lista_palabras_aux = diccionario['palabra']

## Para SPAM

Al igual que con la totalidad de palabras, realizaremos el conteo de las palabras dentro de los mensajes **spam**.

In [52]:
X = conts.fit_transform(spam)

La extracción de características nos devolverá la lista de palabras encontradas dentro de estos mensajes.

In [53]:
pspam=conts.get_feature_names()

Obtenemos la matriz de frecuencia de palabras para los mensajes **spam** y lo almacenamos en un `DataFrame`.

In [55]:
mf_spam=pd.DataFrame(data=X.toarray(),columns=pspam)

Realizamos una iteración de las palabras dentro del diccionario (`lista_palabras_aux`) y, en caso exista la palabra en la matriz de **spam**, obtendremos la sumatoria de la columna. En caso contrario, diremos que la palabra tiene **cero ocurrencias**. Esta lista tendrá el mismo tamaño de la lista de palabras del **diccionario general**, para nuestro ejemplo, 3000 palabras.

In [58]:
mf_spam_actualizado = np.ndarray(dtype=np.int64, shape=(1,len(diccionario)))
for i, palabra in enumerate(lista_palabras_aux):
    if palabra in mf_spam:
        mf_spam_actualizado.itemset(i, sum(mf_spam[palabra]))
    else:
        mf_spam_actualizado.itemset(i, 0)
mf_spam_actualizado = np.ravel(mf_spam_actualizado)

Creamos un DataFrame, al igual que con el diccionario general, para tener mejor visibilidad de las palabras con su ocurrencia.

In [59]:
mf_spam_diccionario = pd.DataFrame({'palabra':np.array(lista_palabras_aux), 'ocurrencia':mf_spam_actualizado})

In [60]:
mf_spam_diccionario

,palabra,ocurrencia
0,number,2483
1,call,283
2,ur,110
3,just,62
4,free,178
5,ok,3
6,ltgt,0
7,know,17
8,day,20
9,ll,3


> **NOTA: Realizaremos el mismo proceso para los mensajes `HAM`**

## Para HAM

In [61]:
conts.fit(ham)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [62]:
pham=conts.get_feature_names()

In [63]:
arr_pham=conts.transform(ham).toarray()

In [64]:
mf_ham=pd.DataFrame(data=arr_pham,columns=pham)

In [65]:
mf_ham_actualizado = np.ndarray(dtype=np.int64, shape=(1,len(diccionario)))

In [66]:
for i, palabra in enumerate(lista_palabras_aux):
    if palabra in mf_ham:
        mf_ham_actualizado.itemset(i, sum(mf_ham[palabra]))
    else:
        mf_ham_actualizado.itemset(i, 0)

In [67]:
mf_ham_actualizado = np.ravel(mf_ham_actualizado)

In [68]:
mf_ham_diccionario = pd.DataFrame({'palabra':np.array(lista_palabras_aux), 'ocurrencia':mf_ham_actualizado})

In [69]:
diccionario[diccionario['palabra']=='number']
mf_spam_actualizado

array([2483,  283,  110, ...,    0,    2,    0], dtype=int64)

In [70]:
mf_ham_diccionario

,palabra,ocurrencia
0,number,987
1,call,168
2,ur,180
3,just,214
4,free,45
5,ok,201
6,ltgt,193
7,know,171
8,day,168
9,ll,180


## Aplicación de la técnica

Hasta el momento hemos realizado los procesos para obtener nuestra data: palabras y su ocurrencia. Ahora realizaremos el algoritmo para obtener las probabilidades que nos servirán para determinar si un mensaje dado es **spam** o **ham**.

In [71]:
a=0.001
l=len(diccionario)
p=sum(mf_ham_actualizado)
probh = np.ndarray(dtype=np.float, shape=(1,l))
for i in range(0,len(mf_ham_actualizado)):
    probh.itemset(i,(mf_ham_actualizado[i]+a)/(p+(a*l)))


In [72]:
p1=sum(mf_spam_actualizado)
probs = np.ndarray(dtype=np.float, shape=(1,l))    
for i in range(0,len(mf_spam_actualizado)):
    probs.itemset(i,(mf_spam_actualizado[i]+a)/(p+(a*l)))


In [73]:
probs=np.ravel(probs)
probh=np.ravel(probh)

In [ ]:
prob_palabras=pd.DataFrame({'Palabras':lista_palabras_aux,'Probabilidad_Spam':probs,'Probabilidad_Ham':probh})

In [ ]:
prob_h=len(mf_ham.index)/(len(mf_ham.index)+len(mf_spam.index))
prob_s=len(mf_spam.index)/(len(mf_ham.index)+len(mf_spam.index))

In [ ]:
def freq(str): 
    str = str.split()          
    str2 = [] 
    for i in str: 
        if i not in str2:
            str2.append(i)  
    for i in range(0, len(str2)):
        str2[i]=(str2[i],str.count(str2[i]))
    return np.array(str2)

In [ ]:
def freq_diccionario(str):
    freq_total = freq(str)
    freq_coincidentes = []
    arr_palabras_diccionario = np.array(diccionario["palabra"])
    for palabra, frecuencia in freq_total:
        if(palabra in arr_palabras_diccionario): # Palabra pertenece al diccionario
            freq_coincidentes.append((palabra, frecuencia))
    return freq_coincidentes

In [ ]:
freq("you should invent your own phrases")

In [ ]:
freq_diccionario("ga")

In [ ]:
freq_diccionario(msj_test[1447])

In [ ]:
def MNB(mensaje):
    nuevo_dic = freq_diccionario(mensaje) 
    probs= [0,0]
    for clase in range(0,2):
        probs[clase] = 0
        for idxd in range (0,len(nuevo_dic)):
            if clase == 0:
                pdp=prob_palabras[prob_palabras.Palabras==nuevo_dic[idxd][0]].Probabilidad_Spam.item()
            if clase == 1:
                pdp=prob_palabras[prob_palabras.Palabras==nuevo_dic[idxd][0]].Probabilidad_Ham.item() 
            power = np.log(1+ int(nuevo_dic[idxd][1])) 
            probs[clase]+=power*np.log(pdp)
        if clase == 0:
            pdc = prob_s
        if clase == 1:
            pdc = prob_h
        probs[clase] +=  np.log(pdc)
    if probs[0] > probs[1]:
        soh='spam'
    if probs[0] < probs[1]:
        soh='ham'
    return soh

In [ ]:
count = 0
equivocados_s = []
equivocados_h = []
for i in msj_test.keys():
    if(MNB(msj_test[i])==etiqueta_test[i]):
        count+=1
    else:
        if(etiqueta_test[i] == 'ham'):
            equivocados_h.append(i)
        else:
            equivocados_s.append(i)

In [ ]:
count

In [ ]:
len(msj_test)

In [ ]:
count/len(msj_test)

In [74]:
# Realiza las conversiones hechas al dataset sobre un mensaje dado
def convertir_string_clasificador(mensaje):
    # PROCESO DE CONVERSIÓN
    df1 = pd.DataFrame({"mensaje":[mensaje]})
    df1 = df1.apply(lambda x: x.astype(str).str.lower()) # To lower case
    df1 = df1['mensaje'].str.replace('http\S+|www.\S+', 'httpaddr', case=False) # Reemplazamos urls
    df1 = df1.replace('\d+', ' number ', regex=True) # Reemplazamos los números
    df1 = df1.replace('[,.;@#?!&$]+', '', regex=True) # Eliminamos los símbolos
    return df1.to_string(index=0)

In [75]:
def MNB_PROB(mensaje):
    convertir_string_clasificador(mensaje)
    nuevo_dic = freq_diccionario(mensaje) 
    probs= [0,0]
    for clase in range(0,2):
        probs[clase] = 0
        for idxd in range (0,len(nuevo_dic)):
            if clase == 0:
                pdp=prob_palabras[prob_palabras.Palabras==nuevo_dic[idxd][0]].Probabilidad_Spam.item()
            if clase == 1:
                pdp=prob_palabras[prob_palabras.Palabras==nuevo_dic[idxd][0]].Probabilidad_Ham.item() 
            power = np.log(1+ int(nuevo_dic[idxd][1])) 
            probs[clase]+=power*np.log(pdp)
        if clase == 0:
            pdc = prob_s
        if clase == 1:
            pdc = prob_h
        probs[clase] +=  np.log(pdc)
    prob_posi = [0, 0]
    prob_posi[0] = -1/probs[0]
    prob_posi[1] = -1/probs[1]
    total_posi = prob_posi[0] + prob_posi[1]
    return [prob_posi[0]/total_posi, prob_posi[1]/total_posi]